In [2]:
import re, os
import shutil
import pickle
from collections import OrderedDict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ROOT_FOLDER = "/mnt/hgfs/projlogs/SFW/sops1/SOPS/Performance Test - Performance_Test1"
ROOT_FOLDER = "/mnt/hgfs/projlogs/SFW/hq1/Performance Test - Performance_Test1"
# MILL, POND, OPP1, OPP2, FOX
# SOPS, HQ1, BAYVIEW, FIELD1
ROOT_FOLDER = "/mnt/hgfs/projlogs/SFW/SOPS"

In [3]:
class TestError():

    def __init__(self, timestamp, userid, cycleid, uj, stepid, errortype, message, test_folder):
        self.time = pd.to_datetime(timestamp, infer_datetime_format=True) 
        self.userid = userid
        self.cycleid = cycleid
        self.stepid = stepid
        self.errortype = errortype
        self.message = message
        self.test_folder = test_folder
        self.uj = uj
        
        self.cause = ''
        self.cause_message = ''
        
        if errortype == 'Success Validation Failure':
            sv = re.search(r'failed sub validations: (.+?)\) for step ".+?" was not found in the response. The response received has been written to "(.+?)"', message)
            self.validation = sv.group(1)
            self.file = sv.group(2)
            # name_parts = self.file.split('__')
            # self.uj = name_parts[0]

        if errortype == 'Response DD Extraction Failure':
            ddisrc = re.search(r'Error reading value for response DD item (.+?) used in step (.+?)', message)
            self.dditem = ddisrc.group(1)
            # !!! the line below currently gives wrong value due to BUG in the ST logs.
            # self.ddi_source = ddisrc.group(2)
    
    def show(self):
        return OrderedDict([('time', self.time),
                ('user', self.userid),
                ('total user errors', 0),
                ('uj', self.uj),
                ('step', self.stepid),
                ('cycle', self.cycleid),
                ('error type', self.errortype),
                ('error cause', self.cause),
                ('loop length', self.loop_len),
                ('loop start', self.loop_start_cycle),
                ('loop end', self.loop_end_cycle),
                ('cause message', self.cause_message),
               ])
    
    def set_snapshot_details(self):
        pass
        
    def _read_error_file(self):
        if self.errortype == 'Success Validation Failure':
            try:
                # for QR, log files are directly in the log folder for the run
                with open(os.path.join(self.test_folder, self.file), 'rt') as f:
                    html = f.read()
            except FileNotFoundError:
                # for non-QR, log files are in subfolders named after virtual user ids
                with open(os.path.join(self.test_folder, 'user'+self.userid , self.file), 'rt') as f:
                    html = f.read()

        return html
    
    def _set_cause(self, cause, message):
        self.cause = cause
        self.cause_message = message
        
    def determine_casuse(self):
        if self.errortype == 'Success Validation Failure':
            html = self._read_error_file()
            if html.count('<label for="username">User Name:</label>')>0:
                self._set_cause('logged off', '')
                return None

            kick_match = re.search(r'redirect><\!\[CDATA\[https*://.+?/maximo/webclient/login/logout.jsp.*?\]\]></redirect>', html)
            if kick_match is not None:
                self._set_cause('Maximo forcefully signed out the user', '')
                return None

            if html.count('title="Please wait...">Please wait...</label>')>0:
                self._set_cause('Long Op', '')
                return None
            
            if html.count("addLongOpTimeout('dolongopquerycheck()',")>0:
                self._set_cause('Long Op', '')
                return None

            if html.count('MessageWarning.png')>0:
                msg = html[html.find('MessageWarning.png'):]
                msg = '<' + msg[:msg.find('</table>')]
                msg = re.sub(r'<[^>]*?>', '', msg)
                msg = msg.replace('\n','').strip()
                self._set_cause('Warning Message', msg)
                return None

            if html.count('st_MessageQuestion.png')>0:
                msg = html[html.find('st_MessageQuestion.png'):]
                msg = '<' + msg[:msg.find('</table>')]
                msg = re.sub(r'<[^>]*?>', '', msg)
                msg = msg.replace('\n','').strip()
                self._set_cause('Question Message', msg)
                return None
            
            if html.count('st_MessageCritical.png')>0:
                msg = html[html.find('st_MessageCritical.png'):]
                msg = '<' + msg[:msg.find('</table>')]
                msg = msg[:msg.find('</component>')]                
                msg = re.sub(r'<[^>]*?>', '', msg)
                msg = msg.replace('\n','').strip()
                self._set_cause('Critical Message', msg)
                return None

            if html.count('>0 - 0 of 0')>0:
                self._set_cause('operation resulted in a table with zero rows', 'searchterm: ' + self.validation)
                return None

            break_index = html.find('---------------Response-----------------')
            html_request_only = html[:break_index]
            html_response_only = html[break_index + 40:]
            if html_response_only.count(self.validation)>0:
                self._set_cause('validation bug', 'searchterm: ' + self.validation)
                return None

            ###  --- COB specific errors ---
            if  html_response_only.count('"id":"0_APPRSS_OPTION","text":"APPRSS"') and not html_response_only.count('Approved'):
                self._set_cause('missing "Approved" option', '')
                return None

            if html_request_only.count('targetId%22%3A%22mx387') and html_response_only.count('title="1 - 2 of 2">1 - 2'):
                self._set_cause('dynamic response', 'missing reference of WO field, thus cant validate')
                return None

            if html_request_only.count('<command>ISWM-RECORDFAILUREREPORT</command>'):
                self._set_cause('response lacks confirmation of recordid', 'response lacks confirmation of recordid')
                return None   
            ### --- ==================== ---

            self._set_cause('unknown validation fail', html)
            self.html = html
        else:
            self._set_cause(self.errortype, '')



    

In [32]:
class Loop():
    
    def __init__(self, lead_error, lenght):
        # the start & end index, should refer to cycle index, not error index
        self.lead_error = lead_error
        self.start = int(lead_error.cycleid)
        self.end = self.start + lenght
        self.lenght = lenght
    
    def __repr__(self):
        return str(self.start) + '-' + str(self.end)

In [33]:
class VirtualUser():
    
    def __init__(self, userid, start_time, uj, errors=[], loops=[], max_loop=None):
        self.userid = userid
        self.start_time = start_time
        self.uj = uj
        self.errors = errors
        self.loops = loops
        self.max_loop = max_loop
        
    # If I can user.errors.append(err) , why create a method?
    def append_error(self, error):
        self.errors.append(error)

    def process_errors(self):
        self.error_count = len(self.errors)
        
        consecutive_count = 0
        consecutivness = ''
        max_consecutive = -1
        max_loop = -1
        # looping over errors in reverse, thus init with values of the last error in the list
        prior_error = self.errors[-1]
        for error in reversed(self.errors):
            error.determine_casuse()

            # the 1st condition needs <= instead of == because there could be multiple errors per cycleid
            # the 2nd condoition is to avoid counting the initial record as a consecutive term
            if int(prior_error.cycleid) - int(error.cycleid) <= 1 and error != self.errors[-1]:
                consecutive_count += 1
            else:
                if consecutive_count > 0:
                    new_loop = Loop(prior_error, consecutive_count)
                    self.loops.append(new_loop)
                    consecutivness = ';' + str(new_loop) + consecutivness

                    # update max_loop
                    if consecutive_count > max_consecutive:
                        max_consecutive = consecutive_count
                        self.max_loop = new_loop

                consecutive_count = 0

            prior_error = error
        
        self._process_cause_stats()
        
        return consecutivness[1:]
    
    def _process_cause_stats(self):
        causes = {}
        for e in self.errors:
            if e.cause in causes.keys():
                causes[e.cause] += 1
            else:
                causes[e.cause] = 1

        self.causes = causes
        

Determine the number of relevant log files in the log's Run folder.

In [34]:
test_folder = ROOT_FOLDER

## Considered running simultaneous analisys of different injectors:
##    it will not work, because user ids are repeated i.e user 0006 on INJ1 will be assigned to UJ1,
##    while user 0006 on INJ2 will have different UJ, and different errors, and etc.

files = [ f for f in os.listdir(test_folder) if os.path.isfile(os.path.join(test_folder,f)) and f.find('.log.')>0 ]
files = [ f for f in files if f.find('lck')==-1 ]
print('at', test_folder, ', found', len(files), 'log files: ', files)

at /mnt/hgfs/projlogs/SFW/SOPS , found 1 log files:  ['SOPS2UA5101XHD_0.log.1']


In case of multiple log files, merge them and work with the merged file.

In [35]:
if len(files)==1:
    FNAME = files[0]
elif len(files)>1:
    print('multiple files - merging into _log.all')
    FNAME = '_log.all'
    with open(os.path.join(test_folder, FNAME), 'wb') as wfd:
        for file in files:
            with open(os.path.join(test_folder, file),'rb') as fd:
                shutil.copyfileobj(fd, wfd, 1024*1024*100)
                #100MB per writing chunk to avoid reading big file into memory.

else:
    if not FNAME:
        print('no files found - please assign FNAME manually')
        exit(1)
    

Obtain the list of errors.

In [36]:
print('processing file: ', test_folder + '/' + FNAME)

with open(test_folder + '/' + FNAME,'r') as f:
    alllog = f.read()

errorlog = re.findall(r'(?<=\n).+?\[SEVERE\].+?id \d+?,[\w\s]+?\):.+?(?=\n)', alllog)
print('severe count:', len(errorlog))

processing file:  /mnt/hgfs/projlogs/SFW/SOPS/SOPS2UA5101XHD_0.log.1
severe count: 6938


Obtain the list of snapshots.

In [37]:
snap = r'<GENERAL_DATA>\n\s+?<APPLICATION>([^<]+?)</APPLICATION>\n\s+?<USER_JOURNEY>([^<]+?)</USER_JOURNEY>\n\s+?<STEP>([^<]+?)</STEP>\n\s+?<ERROR>([^<]+?)</ERROR>\n\s+?<INJECTOR_NAME>([^<]+?)</INJECTOR_NAME>\n\s+?<START_DATE>([^<]+?)</START_DATE>\n\s+?<RUN_NAME>([^<]+?)</RUN_NAME>\n\s+?<BROWSER_PROFILE>([^<]+?)</BROWSER_PROFILE>\n\s+?<DATE>([^<]+?)</DATE>\n\s+?<PROCESS_NAME>([^<]+?)</PROCESS_NAME>\n\s+?<PROCESS_NUMBER>([^<]+?)</PROCESS_NUMBER>\n\s+?<USER_ID>([^<]+?)</USER_ID>\n\s+?<CYCLE_ID>([^<]+?)</CYCLE_ID>\n\s+?<CURRENT_USERS>([^<]+?)</CURRENT_USERS>\n\s+?<CURRENT_CONNECTIONS>([^<]+?)</CURRENT_CONNECTIONS>\n\s+?</GENERAL_DATA>'

snaplog = re.findall(snap, alllog)
print('snapshots:', len(snaplog))

snapshots: 7983


In [38]:
# # Dump the snapshot list into a debug file
# with open(os.path.join(test_folder, 'snaps.dump.txt'), 'wt') as f:
#     f.write('\n'.join(map(str, snaplog)))

Parse the log file.

In [39]:
errors = []
vusers = {}
for err in errorlog:
    time = err[:20]
    usr = re.search(r'user.(\d+?),.cycle', err)
    user = usr.group(1)
    user = str(user).rjust(4,'0')
    cyc = re.search(r'cycle.(\d+?),.step', err)
    cycle = cyc.group(1)
    stp = re.search(r'step\s+?(.+?),\s+?id', err)
    step = stp.group(1)
    errtype = re.search(r', status (.+?)\)', err)
    error_type = errtype.group(1)
    message = err[err.find('):'):]
    
    uj = ''
    try:
        uj = [ z[1] for z in snaplog if z[-4]==str(int(user)) and z[-3]==cycle ][0]
    except IndexError:
        print('failed to find snapshot data for user', user, 'cycle', cycle)
    
    new_error_record = TestError(time, user, cycle, uj, step, error_type, message, test_folder)
    errors.append(new_error_record)
    if user in vusers.keys():
        vusers[user].append_error(new_error_record)
    else:
        vusers[user] = VirtualUser(user, '', uj, [new_error_record])

print('errors:', len(errors), '; erroring_users:', len(vusers))
pickle.dump((errors, vusers), open(os.path.join(test_folder, 'errors_users.pickle'), 'wb' ))

errors: 6938 ; erroring_users: 13


In [40]:
# errors, vusers = pickle.load(open(os.path.join(test_folder, 'errors_users.pickle'), 'rb'))

In [41]:
sorted_keys = sorted(vusers.keys())
print('erroring users', len(sorted_keys))
for userid in sorted_keys:
    user = vusers[userid]
    consecutivness = user.process_errors()
    print('\nuser', user, 'has error count:', user.error_count,
          'and erroring cycles count of:', user.error_cyc_count,
          'out of', user.cycles, 'total cycles')
    print()
    if len(consecutivness):
        print('~~ max error loop:: length', user.max_loop.lenght, 'start cycle:',
              user.max_loop.start, 'end cycle:', user.max_loop.end)
        print('~~', consecutivness)

pickle.dump(vusers, open(os.path.join(test_folder, 'with_loops.pickle'), 'wb' ))

erroring users 13


AttributeError: 'VirtualUser' object has no attribute 'error_cyc_count'

In [12]:
# vusers = pickle.load(open(os.path.join(test_folder, 'with_loops.pickle'), 'rb'))

In [13]:
def show_more(user):
    max_loop = {'loop_len': '', 'loop_start': '', 'loop_end': ''}
    try:
        e = vusers[user][int(loops[user])]
#         print(loops[user], e.show())
        max_loop = {'loop_len': e.loop_len, 'loop_start': e.loop_start_cycle, 'loop_end': e.loop_end_cycle}
#         try:
#             max_loop = {'loop_len': e.loop_len, 'loop_start': e.loop_start_cycle, 'loop_end': e.loop_end_cycle}
#         except AttributeError: # is this needed?
#             pass
    except KeyError:
        print('no consequtive errors detected for:', user, '; reporting on first error instead')
        e = vusers[user][0]
        
    msg = ''
    #if e.cause in ['Warning Message', 'Critical Message', 'Question Message', 'dynamic response']:
    #    msg = e.cause_message
    error_details = e.show()
#     print(error_details)
    error_details['total user errors'] += len(vusers[user])
    error_details.update(max_loop)
    
    return e, error_details

In [14]:
causes = {}
causes_keys = []
accu = []

for userid in sorted_keys:
    user = vusers[userid]
    
    if user.max_loop:
        accu.append(dict(user.max_loop.lead_error.show()))
    
    # summation of values of corresponding key over list of dicts
    for key in user.causes.keys():
        causes[key]=causes.setdefault(key, 0) + user.causes[key]
    
print('count of erroring users:', len(sorted_keys))
for k,v in causes.items():
    print('cause', k, 'has count', v)
    

cols = ['time', 'user', 'total user errors', 'uj', 'cycle', 'error type',
        'error cause', 'loop_len', 'loop_start', 'loop_end', 'cause message',]

accu = [ [ z[y] for y in cols ] for z in accu ]
    
data = pd.DataFrame(accu, columns=cols)

no consequtive errors detected for: 0006 ; reporting on first error instead
no consequtive errors detected for: 0007 ; reporting on first error instead
count of erroring users: 13
cause Response DD Extraction Failure has count 7
cause Long Op has count 1
cause Critical Message has count 3
cause operation resulted in a table with zero rows has count 2


In [15]:
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 999
data

,time,user,total user errors,uj,cycle,error type,error cause,loop_len,loop_start,loop_end,cause message
0,2018-01-29 06:08:30,0000,1586,Open and Change to COMP as Supervisor,199,Response DD Extraction Failure,Response DD Extraction Failure,1583,199,1782,
1,2018-01-29 06:10:41,0001,1568,Open and Change to COMP as Supervisor,201,Response DD Extraction Failure,Response DD Extraction Failure,1562,201,1763,
2,2018-01-29 06:10:33,0002,1571,Open and Change to COMP as Supervisor,201,Response DD Extraction Failure,Response DD Extraction Failure,1566,201,1767,
3,2018-01-29 02:59:17,0006,1,Advanced Search of WorkOrder by Asset,23,Response DD Extraction Failure,Response DD Extraction Failure,,,,
4,2018-01-29 06:08:35,0007,2,Advanced Search of WorkOrder by Asset,76,Success Validation Failure,Long Op,,,,
5,2018-01-29 06:34:37,0008,342,Open and Assign CM WO as Supervisor,197,Response DD Extraction Failure,Response DD Extraction Failure,320,197,517,
6,2018-01-29 06:34:15,0009,337,Open and Assign CM WO as Supervisor,196,Response DD Extraction Failure,Response DD Extraction Failure,320,196,516,
7,2018-01-29 05:11:59,0010,221,open edit SR and Convert to WO,149,Success Validation Failure,Critical Message,219,149,368,"BMXAA7837E - An error occured that prevented the GISCREATEUPDATESRWOPOINT script for the GISCREATEUPDSRWPT_OLP launch point from running.\tjava.lang.Exception: java.lang.Exception: Unable to copy Maximo attributes to GIS feature.Traceback &#040;most recent call last&#041;: File: &quot;&lt;script&gt;&quot;, line 1102, in &lt;module&gt;MXSystemException: BMXAA4211E - Database error number 1000 has occurred when operating on WOPOINT : Objectid=1,038,311. Report the error to the owner of the deployment. in &lt;script&gt; at line number 1105"
8,2018-01-29 05:06:26,0011,223,open edit SR and Convert to WO,146,Success Validation Failure,Critical Message,221,146,367,"BMXAA7837E - An error occured that prevented the GISCREATEUPDATESRWOPOINT script for the GISCREATEUPDSRPT_OLP launch point from running.\tjava.lang.Exception: java.lang.Exception: Unable to copy Maximo attributes to GIS feature.Traceback &#040;most recent call last&#041;: File: &quot;&lt;script&gt;&quot;, line 1102, in &lt;module&gt;MXSystemException: BMXAA4211E - Database error number 1000 has occurred when operating on SRPOINT : Objectid=404,496. Report the error to the owner of the deployment. in &lt;script&gt; at line number 1105"
9,2018-01-29 06:34:09,0012,137,Open WO finding and FIELDCOMP,96,Success Validation Failure,operation resulted in a table with zero rows,97,96,193,"searchterm: value=""No Follow Up Action was Required"""


In [16]:
[ z.show() for z in vusers['0012'] ]

[{'cause message': 'BMXAA8229W - Record WORKORDER :  Site=WW Work Order=4879992 has been updated by another user. Your changes have not been saved. Refresh the record and try again.',
  'cycle': '19',
  'error cause': 'Warning Message',
  'error type': 'Success Validation Failure',
  'time': Timestamp('2018-01-29 02:51:13'),
  'total user errors': 0,
  'uj': 'Open WO finding and FIELDCOMP',
  'user': '0012'},
 {'cause message': 'BMXAA2256E - Field Additional Findings is read-only.',
  'cycle': '22',
  'error cause': 'Critical Message',
  'error type': 'Success Validation Failure',
  'time': Timestamp('2018-01-29 03:00:55'),
  'total user errors': 0,
  'uj': 'Open WO finding and FIELDCOMP',
  'user': '0012'},
 {'cause message': 'BMXAA8229W - Record WORKORDER :  Site=WW Work Order=4880255 has been updated by another user. Your changes have not been saved. Refresh the record and try again.',
  'cycle': '23',
  'error cause': 'Warning Message',
  'error type': 'Success Validation Failure',

In [ ]:
def plot_user_errors(user):
    as_np_array = np.array([ z.show_as_list() for z in vusers[user] ])
    user_df = pd.DataFrame(as_np_array, columns=['time', 'user', 'cycle', 'uj', 'err_type', 'err_cause',
                                   'total_user_errors', 'err_cause_message', ])
    per_hr = user_df.groupby(user_df['time'].dt.hour)
    per_hr['user'].count().plot('bar', color='b');

In [ ]:
plot_user_errors('0014')

In [ ]:
[ z.show() for z in vusers['0014'] ]

In [ ]:
plot_user_errors('0008')

In [ ]:
[ z.show() for z in vusers['0008'] ]

In [ ]:
plot_user_errors('0009')

status to COMP as Supervisor

In [ ]:
[ z.show() for z in vusers['0001'] ]

Open, edit a SR and Convert to WO

In [ ]:
plot_user_errors('0011')

In [17]:
[ z.show() for z in vusers['0011'] ]

[{'cause message': 'BMXAA7837E - An error occured that prevented the GISCREATEUPDATESRWOPOINTSA script for the GISCREATEUPDATESRWOPOINTWO_OLP launch point from running.\tpsdi.util.MXApplicationException: psdi.util.MXApplicationException: BMXAA7837E - An error occured that prevented the GISCREATEUPDATESRWOPOINT script for the GISCREATEUPDSRWPT_OLP launch point from running.\tjava.lang.Exception: java.lang.Exception: Unable to copy Maximo attributes to GIS feature.Traceback &#040;most recent call last&#041;:  File: &quot;&lt;script&gt;&quot;, line 1102, in &lt;module&gt;MXSystemException: BMXAA4211E - Database error number 1000 has occurred when operating on WOPOINT :  Objectid=1,038,290. Report the error to the owner of the deployment. in &lt;script&gt; at line number 1105 in &lt;script&gt; at line number 50',
  'cycle': '145',
  'error cause': 'Critical Message',
  'error type': 'Success Validation Failure',
  'time': Timestamp('2018-01-29 05:05:01'),
  'total user errors': 0,
  'uj': 